In [71]:
import json
import os

import cv2
import mediapipe as mp

In [72]:
if not os.path.exists('video_in'):
    os.mkdir('video_in')

if not os.path.exists('landmarks'):
    os.mkdir('landmarks')

In [73]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    model_complexity=1,
    min_detection_confidence=.9,
    min_tracking_confidence=.9,
)

In [74]:
def get_image_landmarks(image):
    mp_result = hands.process(image)

    # If hand not detected, retry, this time without
    # tracking landmarks from previous image
    if not mp_result.multi_hand_landmarks:
        mp_result = hands.process(image)

    landmarks = mp_result.multi_hand_landmarks[0].landmark

    return [(lm.x, lm.y, lm.z) for lm in landmarks[0:9]]

In [75]:
def get_video_info(video_path):
    vc = cv2.VideoCapture(video_path)

    ret = {
        'width': int(vc.get(cv2.CAP_PROP_FRAME_WIDTH)),
        'height': int(vc.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        'frames': int(vc.get(cv2.CAP_PROP_FRAME_COUNT)),
        'fps': vc.get(cv2.CAP_PROP_FPS),
        'fourcc': int(vc.get(cv2.CAP_PROP_FOURCC)),
    }

    vc.release()
    return ret

def read_video(video_path):
    vc = cv2.VideoCapture(video_path)
    is_portrait = vc.get(cv2.CAP_PROP_FRAME_WIDTH) < vc.get(cv2.CAP_PROP_FRAME_HEIGHT)
    while vc.grab():
        bgr = vc.retrieve()[1]
        if is_portrait:
            bgr = cv2.rotate(bgr, cv2.ROTATE_180)
        yield cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    vc.release()

In [76]:
def get_video_landmarks(video_path):
    video = read_video(video_path)
    video_info = get_video_info(video_path)
    video_landmarks = []

    for frame_number, image in enumerate(video):
        video_landmarks.append({
            'time': frame_number / video_info['fps'],
            'landmarks': get_image_landmarks(image)
        })

    return video_landmarks

In [77]:
for file in os.listdir('video_in'):
    video_landmarks = get_video_landmarks(f'video_in/{file}')
    file_name = os.path.splitext(file)[0]
    with open(f'landmarks/{file_name}.json', 'w') as outfile:
        json.dump(video_landmarks, outfile)